# Clean Street Quality Data for Merge

In [24]:
# Import geojson file into geopandas dataframe and remove streets with no rating
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

st_quality = gpd.read_file('../data/external/StreetAssessmentRating/StreetAssessmentRating.shp')
st_quality = st_quality[['SegmentID', 'Rating_B', 'geometry']]
st_quality = st_quality[st_quality.Rating_B != 0 ]
st_quality = st_quality.to_crs({'init' :'epsg:4326'})
st_quality.head()

,SegmentID,Rating_B,geometry
5,15,8,LINESTRING (-74.25505718173595 40.505211202870...
6,16,8,LINESTRING (-74.25422295126857 40.506165510159...
7,17,8,LINESTRING (-74.25317481600673 40.504767963667...
8,20,7,LINESTRING (-74.2530943012904 40.5062970118639...
9,25,6,LINESTRING (-74.25493722035228 40.507701030695...


In [25]:
# Import citibike station csv into geopandas dataframe
from shapely.geometry import Point
from geopandas import GeoDataFrame
from geopandas import GeoSeries

stations = pd.read_csv('../data/external/citibike_station_info.csv')

geometry = GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
geometry = geometry.buffer(.0005)
geo_stations = GeoDataFrame(stations, geometry=geometry)
geo_stations.crs = {'init' :'epsg:4326'}
geo_stations.head()
geo_stations.to_file('geo_stations')

In [26]:
# Merge street quality data with citibike stations using Geopandas Spatial Merge
stations_st_quality = gpd.sjoin(geo_stations, st_quality, how="inner", op='intersects')
stations_st_quality.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,index_right,SegmentID,Rating_B
0,72,W 52 St & 11 Ave,W 52 St & 11 Ave,40.767272,-73.993929,"POLYGON ((-73.99342888 40.76727216, -73.993431...",26239,33779,8
0,72,W 52 St & 11 Ave,W 52 St & 11 Ave,40.767272,-73.993929,"POLYGON ((-73.99342888 40.76727216, -73.993431...",26241,33781,8
0,72,W 52 St & 11 Ave,W 52 St & 11 Ave,40.767272,-73.993929,"POLYGON ((-73.99342888 40.76727216, -73.993431...",26240,33780,8
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",24501,31767,9
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",24511,31777,8
